In [1]:
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D

import keras 
import os 

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


In [2]:
!tree

.
├── Cancer\ Cells\ Classification.ipynb
└── data
    ├── train
    │   ├── negative
    │   │   ├── 1_21_10_2_7_1004_ec6865d8f647e36c9c3df06ed00f353ab4de475ede5006580e4b8bec87b96b27.jpg
    │   │   ├── 1_21_10_3_7_1004_603d88abd817870b9869cc4677ffbf23d1eb13768d8ba27742e87ca30212a5ec.jpg
    │   │   ├── 1_21_10_4_7_1004_90bc8e78fc653a04493f3df202f0d7ea863b795119d702106ae952f30437f29e.jpg
    │   │   ├── 1_21_10_6_7_1004_4f92ea2cd08a6fb88f495f17b9f9883f0b14143bdd923518b6fc0aecb193e89a.jpg
    │   │   ├── 1_21_11_5_7_1004_19de4ebed43e327354f1f614ae7d4ff5c74639eb10bdb97d7c33db42cf467b7b.jpg
    │   │   ├── 1_21_12_1_7_1004_7bd6c76f3ec4cc6222e246f3e5641e7ab4fc8a9ac1ce01daf314a848b0595350.jpg
    │   │   ├── 1_21_12_3_7_1004_92bb7904a0ebe04f9b148fa047e239cb91176820a890edf789d97c38d396b502.jpg
    │   │   ├── 1_21_12_4_7_1004_350b53e98f985d9f7fbf32f4a542056eb1b1786283ea23bb1411258c8d5bb485.jpg
    │   │   ├── 1_21_12_5_7_1004_002ba356a84d9af59cc464959c8d1e9da3e241bbab1ea0432db08679c112b99c.

In [3]:
image_paths_dict = dict(train = list(), 
                  val = list())

for subdir in os.listdir('data/'): 
    
    if subdir in image_paths_dict:
        print(subdir)
        for cat in os.listdir('data/' + subdir):
            if cat[0] != ".": # Exclude hidden files 
                for img_path in os.listdir('data/' + subdir + '/' + cat):
                    full_img_path = f"./{subdir}/{cat}/{img_path}"
                    
                    #create a dictionary representing the image path to be placed in the dataframe 
                    obj_rep = dict(target = cat, image_path = full_img_path)
                    image_paths_dict[subdir].append(obj_rep)
            
df_train = pd.DataFrame(image_paths_dict["train"]) 
df_val = pd.DataFrame(image_paths_dict["val"]) 

train
val


In [4]:
print("Training Data Set")
print(df_train.target.value_counts())

print("Validation Data Set")
print(df_val.target.value_counts())

Training Data Set
negative    219
positive    117
Name: target, dtype: int64
Validation Data Set
negative    103
positive     64
Name: target, dtype: int64


In [5]:
df_train.head()

,image_path,target
0,./train/positive/2_22_2_2_8_1004_84191a6b28e03...,positive
1,./train/positive/1_21_3_5_7_1004_394bfa70a9bae...,positive
2,./train/positive/2_22_12_1_8_1004_4db399ebaa3f...,positive
3,./train/positive/4_21_2_7_7_1004_2bca783196104...,positive
4,./train/positive/2_22_10_2_8_1004_a162a0b14566...,positive


In [6]:
df_train.tail()

,image_path,target
331,./train/negative/1_22_11_7_8_1004_f831cbd16a4d...,negative
332,./train/negative/3_22_2_4_8_1004_bc646499ac5fa...,negative
333,./train/negative/2_22_6_2_8_1004_0ab143fb6ec79...,negative
334,./train/negative/1_21_11_5_7_1004_19de4ebed43e...,negative
335,./train/negative/1_22_4_3_8_1004_5c66f3548e9d8...,negative


In [7]:
os.listdir('data/train')

['positive', '.DS_Store', 'negative']

In [18]:
default_model = VGG16(weights = 'imagenet', include_top = False)

x = default_model.output 
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [19]:
model=Model(inputs=default_model.input, outputs = predictions)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Freezing the convolutional layers to focus the training on the newly added dense layers

In [20]:
for i, layer in enumerate(model.layers):
    if i < 19: 
        layer.trainable = False
        
    else: 
        layer.trainable = True
        
    print(f"Layer {i}, {layer.name}, Trainable: {layer.trainable}")

Layer 0, input_2, Trainable: False
Layer 1, block1_conv1, Trainable: False
Layer 2, block1_conv2, Trainable: False
Layer 3, block1_pool, Trainable: False
Layer 4, block2_conv1, Trainable: False
Layer 5, block2_conv2, Trainable: False
Layer 6, block2_pool, Trainable: False
Layer 7, block3_conv1, Trainable: False
Layer 8, block3_conv2, Trainable: False
Layer 9, block3_conv3, Trainable: False
Layer 10, block3_pool, Trainable: False
Layer 11, block4_conv1, Trainable: False
Layer 12, block4_conv2, Trainable: False
Layer 13, block4_conv3, Trainable: False
Layer 14, block4_pool, Trainable: False
Layer 15, block5_conv1, Trainable: False
Layer 16, block5_conv2, Trainable: False
Layer 17, block5_conv3, Trainable: False
Layer 18, block5_pool, Trainable: False
Layer 19, global_average_pooling2d_2, Trainable: True
Layer 20, dense_11, Trainable: True
Layer 21, dense_12, Trainable: True
Layer 22, dense_13, Trainable: True
Layer 23, dense_14, Trainable: True
Layer 24, dense_15, Trainable: True


In [21]:
# preprocess_input helps to do data augmentation

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./data/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 336 images belonging to 2 classes.


In [22]:
val_datagen= ImageDataGenerator(rescale =1./255)
val_generator = val_datagen.flow_from_directory('./data/val',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 167 images belonging to 2 classes.


In [23]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
step_size_val =val_generator.n//val_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=val_generator,
                    validation_steps=step_size_val,
                    epochs=10)



Epoch 1/10
10/10 [==============================] - 287s 29s/step - loss: 1.0689 - acc: 0.7094 - val_loss: 0.6795 - val_acc: 0.6312
Epoch 2/10
10/10 [==============================] - 251s 25s/step - loss: 0.3592 - acc: 0.8530 - val_loss: 0.6825 - val_acc: 0.5926
Epoch 3/10
10/10 [==============================] - 253s 25s/step - loss: 0.3175 - acc: 0.8596 - val_loss: 0.6874 - val_acc: 0.5630
Epoch 4/10
10/10 [==============================] - 234s 23s/step - loss: 0.2760 - acc: 0.8653 - val_loss: 0.6377 - val_acc: 0.6741
Epoch 5/10
10/10 [==============================] - 279s 28s/step - loss: 0.2611 - acc: 0.8807 - val_loss: 0.6580 - val_acc: 0.6370
Epoch 6/10
10/10 [==============================] - 264s 26s/step - loss: 0.2437 - acc: 0.8747 - val_loss: 0.6774 - val_acc: 0.6000
Epoch 7/10
10/10 [==============================] - 266s 27s/step - loss: 0.2124 - acc: 0.8901 - val_loss: 0.6686 - val_acc: 0.6188
Epoch 8/10
10/10 [==============================] - 225s 22s/step - loss: 0.

In [14]:
!pip3 install pillow


In [15]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [16]:
from PIL import Image
print(Image.__file__)

/usr/local/lib/python3.7/site-packages/PIL/Image.py


In [17]:
import Image
print(Image.__file__)

/usr/local/lib/python3.7/site-packages/PIL/Image.py
